In [1]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import upfirdn, convolve, welch
import ipywidgets as widgets
from IPython.display import display, clear_output


def rtrapezium(nsamp, rolloff, delay):
    T = 1
    t = np.arange(-delay*T, (delay*T) + 1/nsamp, 1/nsamp)
    rrc = np.zeros_like(t)
    for i, ti in enumerate(t):
        if ti == 0.0:
            rrc[i] = 1.0 - rolloff + 4*rolloff/np.pi
        elif abs(ti) == T / (4 * rolloff):
            rrc[i] = (rolloff / np.sqrt(2)) * (((1 + 2/np.pi) * (np.sin(np.pi / (4 * rolloff)))) + ((1 - 2/np.pi) * (np.cos(np.pi / (4 * rolloff)))))
        else:
            rrc[i] = (np.sin(np.pi * ti * (1 - rolloff) / T) + 4 * rolloff * ti * np.cos(np.pi * ti * (1 + rolloff) / T)) / (np.pi * ti * (1 - (4 * rolloff * ti / T) ** 2))
    return rrc / np.sqrt(np.sum(rrc**2))

def run_simulation(f1, f2, qam_type):
    with output8:
        clear_output(wait=True)

        # Parameters
        k = int(np.log2(qam_type))
        M = 2**k
        Nsymb = 30000
        pulse_type = 1  # 1 for rtrapezium shaping filter, 0 for rectangular pulse
        nsamp = 32  # oversampling factor
        fc = (f1 + f2) / 2  # carrier frequency
        bandwidth = f2 - f1  # signal bandwidth
        rolloff = bandwidth / (2 * fc)  # adjust rolloff factor based on bandwidth
        EbNo = 10  # Eb/No in dB
        SNR = EbNo - 10 * np.log10(nsamp / k / 2)  # SNR per signal sample

        # Phase and mapping initialization
        ph1 = np.pi / 4
        theta = np.array([ph1, -ph1, np.pi - ph1, -np.pi + ph1])
        mapping = np.exp(1j * theta)

        if k > 2:
            for j in range(3, k + 1):
                theta = theta / 2
                mapping = np.exp(1j * theta)
                mapping = np.concatenate([mapping, -np.conj(mapping)])
                theta = np.angle(mapping)

        # Transmitter
        x = np.random.randint(0, 2, k * Nsymb)  # random binary sequence
        xsym = x.reshape(-1, k)
        xsym = xsym.dot(2**np.arange(xsym.shape[-1])[::-1])  # bitwise to decimal
        y = mapping[xsym]

        # Shaping filter definition
        if pulse_type == 1:  # Nyquist pulse -- rtrapezium
            delay = 8  # Group delay (# of T periods)
            shaping_filter = rtrapezium(nsamp, rolloff, delay)
        else:  # Rectangular pulse
            delay = 0.5
            shaping_filter = np.ones(nsamp) / np.sqrt(nsamp)  # with normalization

        # Transmitted signal
        ytx = upfirdn([1], y, nsamp)
        ytx = convolve(ytx, shaping_filter, mode='same')

        # Quadrature modulation
        m = np.arange(len(ytx))
        s = np.real(ytx * np.exp(1j * 2 * np.pi * fc * m / nsamp))

        # Adding white Gaussian noise
        Ps = 10 * np.log10(np.mean(s**2))  # signal power in dB
        Pn = Ps - SNR  # corresponding noise power in dB
        n = np.sqrt(10**(Pn / 10)) * np.random.randn(len(ytx))
        snoisy = s + n  # noisy bandpass signal

        # Receiver
        yrx = 2 * snoisy * np.exp(-1j * 2 * np.pi * fc * m / nsamp)
        yrx = convolve(yrx, shaping_filter, mode='same')

        # Spectrum plot of received signal
        f, Pxx_den = welch(np.real(s), fs=nsamp, nperseg=1024)
        Pxx_den = 10 * np.log10(Pxx_den)
        plt.figure(figsize=(10, 8))
        plt.plot(f, Pxx_den, 'r')
        plt.title('Welch Power Spectral Density Estimate')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power Spectral Density (V^2/Hz)')
        plt.xlim(0, max(f1, f2) + 5)
        plt.grid()
        plt.show()

# Widgets for f1, f2, and QAM type
f1_widget = widgets.FloatText(value=6.75, description='f1:')
f2_widget = widgets.FloatText(value=9.25, description='f2:')
qam_widget = widgets.Dropdown(options=[4, 16, 64], value=16, description='QAM Type:')

def on_value_change(change):
    run_simulation(f1_widget.value, f2_widget.value, qam_widget.value)

f1_widget.observe(on_value_change, names='value')
f2_widget.observe(on_value_change, names='value')
qam_widget.observe(on_value_change, names='value')

output8 = widgets.Output()

display(qam_widget,f1_widget, f2_widget, output8)

# Run initial simulation
run_simulation(f1_widget.value, f2_widget.value, qam_widget.value)

Dropdown(description='QAM Type:', index=1, options=(4, 16, 64), value=16)

FloatText(value=6.75, description='f1:')

FloatText(value=9.25, description='f2:')

Output()